#Instalação de bibliotecas

In [15]:
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install piglet

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package python-opengl
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [16]:
!pip install cmake 'gym[atari]' scipy --quiet

#Importação de bibliotecas

In [17]:
import gym
from IPython import display
import matplotlib.pyplot as plt
from IPython.display import clear_output
from time import sleep
from tqdm import tqdm
import numpy as np
import random
from IPython.display import clear_output
%matplotlib inline

#instanciando ambiente da OpenAI


Instanciando um ambiente utilizando a biblioteca gym da OpenAI. a bilbioteca gym disponibiliza ambientes pré-configurados para treinar algoritmos de aprendizado por reforço

In [18]:
env = gym.make('Taxi-v3').env

o ambiente permite 6 ações possíveis: 'south', 'north', 'east', 'west', 'pickup', e 'dropoff'.

In [19]:
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space Discrete(6)
State Space Discrete(500)


#Execução do agente com uma política aleatória

resetando o ambiente, setando um estado inicial e criando um array para armazenar os frames

In [20]:
def runRandom():
  env.reset()
  env.s = 328
  epochs = 0
  penalties, reward = 0, 0
  frames = []

  done = False

  while not done:
      action = env.action_space.sample()
      state, reward, done, info = env.step(action)

      if reward == -10:
          penalties += 1

      frames.append({
          'frame': env.render(mode='ansi'),
          'state': state,
          'action': action,
          'reward': reward
          }
      )

      epochs += 1
  return epochs, penalties, frames

In [21]:
epochsRandom = []
penaltiesRandom = []
framesRandom = []

In [22]:
for _ in tqdm(range(100)):
  epochs, penalties, frames = runRandom()
  epochsRandom.append(epochs)
  penaltiesRandom.append(penalties)
  framesRandom.append(frames)

100%|██████████| 100/100 [00:38<00:00,  2.60it/s]


In [23]:
def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)

#Implementando o algoritmo Q-learning

Executando o algoritmo Q-learning

In [24]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [25]:
%%time
"""Training the agent"""

alpha = 0.1
gamma = 0.6
epsilon = 0.1

all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False

    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state])

        next_state, reward, done, info = env.step(action)

        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])

        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1

    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: user 1min 22s, sys: 5.99 s, total: 1min 28s
Wall time: 1min 37s


In [26]:
q_table[328]

array([ -2.40883216,  -2.27325184,  -2.41221126,  -2.35874332,
       -10.59716512, -10.42627212])

#Avaliando a performance do agente com a política alteatória e com a política e com a política épsilon-greedy após q-learning.

In [27]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0

    done = False

    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

In [28]:
print(f"Após {episodes} episódeos:")
print(f"Média de épocas por episódeo com Q-learning: {total_epochs / episodes}")
print(f"Média de épocas por episódeo sem Q-learning: {sum(epochsRandom) / len(epochsRandom)}")
print(f'Razão: {round((sum(epochsRandom) / len(epochsRandom)) / (total_epochs / episodes),0 )} vezes mais rápido')

Após 100 episódeos:
Média de épocas por episódeo com Q-learning: 12.92
Média de épocas por episódeo sem Q-learning: 2394.07
Razão: 185.0 vezes mais rápido
